In [3]:
import torch
import numpy as np
from PIL import Image
from torchvision.transforms.functional import to_pil_image
from facenet_pytorch.models.mtcnn import PNet, RNet, ONet  # Import P-Net, R-Net, O-Net
from facenet_pytorch import MTCNN
import os
import matplotlib.pyplot as plt
from facenet_pytorch import InceptionResnetV1
import cv2
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn.functional import normalize



c:\Users\harsh.n\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
match_images_folder = r'D:\Images\Facial_reco_images\Facial_reco_images\Game'  # match photos
studio_images_folder = r'E:\2024\hires\324\Game\363155\2024-JUNE 13-SATURDAY-TIME 4-FIELD 4-324-GAME-363155-SCHEDULE-249.JPG'  # studio photos

In [5]:
# Define fine-tuned P-Net, R-Net, and O-Net for finetuning
class FinetunedMTCNN(MTCNN):
#(self, image_size=160, margin=5, **kwargs): # use this after some time to improve the final results
    def __init__(self, **kwargs):
        super(FinetunedMTCNN, self).__init__(**kwargs)
        # Create your custom, finetuned P-Net, R-Net, O-Net here
        self.pnet = PNet()
        self.rnet = RNet()
        self.onet = ONet()
    def forward(self, x):
        # Overriding forward pass if additional finetuning is needed
        return super().forward(x)

In [6]:
# Initialize FaceNet model
facenet = InceptionResnetV1(pretrained='vggface2').eval()  # Load the FaceNet model
# Initialize mtcnn model
finetuned_mtcnn = FinetunedMTCNN(keep_all=True, device='cuda:0' if torch.cuda.is_available() else 'cpu', min_face_size=60)

In [7]:
def find_euclidean_distance(src, dst):
    return np.linalg.norm(src - dst)

In [8]:
def alignment_procedure(img, left_eye, right_eye):    
    left_eye_x, left_eye_y = left_eye
    right_eye_x, right_eye_y = right_eye
    
    # Find the direction to rotate the image based on the eye coordinates
    if left_eye_y > right_eye_y:
        point_3rd = (right_eye_x, left_eye_y)
        direction = -1  # Clockwise
    else:
        point_3rd = (left_eye_x, right_eye_y)
        direction = 1  # Counter-clockwise
    
    # Calculate the length of the triangle edges
    a = find_euclidean_distance(np.array(left_eye), np.array(point_3rd))
    b = find_euclidean_distance(np.array(right_eye), np.array(point_3rd))
    c = find_euclidean_distance(np.array(left_eye), np.array(right_eye))
    
    # Apply cosine rule to find the angle
    if b != 0 and c != 0:  # Avoid division by zero
        cos_a = (b**2 + c**2 - a**2) / (2 * b * c)
        angle = np.arccos(cos_a)  # Angle in radians
        angle = np.degrees(angle)  # Convert to degrees
        
        # Adjust the angle based on the rotation direction
        if direction == -1:
            angle = 90 - angle
        
        # Rotate the image using PIL
        #img = Image.fromarray(img)
        img = img.rotate(direction * angle, resample=Image.BICUBIC)
        img = np.array(img)  # Convert back to numpy array
        # aligned_img_pil = Image.fromarray(img)  # Convert back to PIL Image for saving
        # aligned_img_pil.show()
    return img

In [9]:
# Function to perform face detection and store image paths with their cropped face regions
def process_and_save_match_faces(image_folder, mtcnn_model, margin=0):
    global match_photos_all
    match_photos_all = []
    match_photos_paths = []
    image = Image.open(image_folder)
    boxes, confidences, landmarks = mtcnn_model.detect(image, landmarks=True)
    if boxes is not None:
        # Set a confidence threshold
        threshold = 0.95
        # Filter detected faces based on the confidence score
        filtered_faces = [i for i, confidence in enumerate(confidences) if confidence > threshold]
        #print(f"Number of filtered faces: {len(filtered_faces)}")
        # Process each filtered face
        for i in filtered_faces:
            box = boxes[i]  # Get the bounding box for the filtered face
            box = [int(b) for b in box]  # Ensure the box is in integer format
            # adding margin around the box
            # Apply margin to the bounding box
            x1 = max(0, box[0] - margin)  # Left
            y1 = max(0, box[1] - margin)  # Top
            x2 = min(image.width, box[2] + margin)  # Right
            y2 = min(image.height, box[3] + margin)  # Bottom
            # Crop the face from the image
            cropped_face = image.crop((x1, y1, x2, y2))
            if cropped_face is not None:
                # Get the landmarks (left and right eyes) for the current face
                left_eye = landmarks[i][0]  # Left eye coordinates for face i
                right_eye = landmarks[i][1]  # Right eye coordinates for face i
                # Align the cropped face using the eye coordinates
                aligned_face = alignment_procedure(cropped_face, left_eye, right_eye)
                # Store the aligned face and the original image path
                match_photos_all.append(aligned_face)
                match_photos_paths.append(image_folder)  # Store original image path
    return match_photos_paths  # Return list of original image paths

In [15]:
def process_and_save_studio_faces(image_folder, mtcnn_model, margin=0):
    global studio_photos_all
    studio_photos_all = []
    studio_photos_paths = []
    image = Image.open(image_folder)
    # Detect faces (with bounding boxes, probabilities, and landmarks)
    boxes, confidences, landmarks = mtcnn_model.detect(image, landmarks=True)
    if boxes is not None:
        # Set a confidence threshold
        threshold = 0.95
        # Filter detected faces based on the confidence score
        filtered_faces = [i for i, confidence in enumerate(confidences) if confidence > threshold]
        # Save original image paths and face regions that pass the confidence threshold
        for i in filtered_faces:
            box = boxes[i]  # Get the bounding box for the filtered face
            box = [int(b) for b in box]  # Ensure the box is in integer format
            # Apply margin to the bounding box
            x1 = max(0, box[0] - margin)  # Left
            y1 = max(0, box[1] - margin)  # Top
            x2 = min(image.width, box[2] + margin)  # Right
            y2 = min(image.height, box[3] + margin)  # Bottom
            # Crop the face from the image
            cropped_face = image.crop((x1, y1, x2, y2))
            if cropped_face is not None:
                left_eye = landmarks[i][0]
                #print(left_eye)
                right_eye = landmarks[i][1]
                #print(right_eye)
                aligned_face = alignment_procedure(cropped_face, left_eye, right_eye)
                studio_photos_all.append(aligned_face)
                studio_photos_paths.append(image_folder)  # Store original image path
    return studio_photos_paths  # Return list of original image paths


In [16]:
def get_face_embedding(image):
    image = cv2.resize(image, (160, 160))
    image = torch.tensor(image).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    embedding = facenet(image)
    embedding = normalize(embedding, p=2, dim=1)  # L2 normalization of embeddings
    return embedding.detach().numpy()

In [17]:
# Function to calculate Cosine Similarity between embeddings
def cosine_similarity_metric(embedding1, embedding2):
    return cosine_similarity(embedding1.reshape(1, -1), embedding2.reshape(1, -1))[0][0]

In [18]:
# Process both match and studio images, and return original image paths
studio_photos_paths = process_and_save_studio_faces(studio_images_folder, finetuned_mtcnn, margin=200)
studio_image_shapes = [np.array(image).shape for image in studio_photos_all]
print("Shapes of images in the list studio_photos_all:", studio_image_shapes)
print(f"Number of studio photos: {len(studio_photos_all)}")

Shapes of images in the list studio_photos_all: [(634, 574, 3)]
Number of studio photos: 1


In [19]:
# Generate embeddings for studio photos
studio_embeddings = []
for i, studio_photo in enumerate(studio_photos_all):
    studio_image = np.array(studio_photo)  # Convert the PIL image to a NumPy array
    embedding = get_face_embedding(studio_image)  # Get embedding
    if embedding is not None:
        studio_embeddings.append((i, embedding))  # Store the index and embedding

In [ ]:
print(studio_embeddings[0])

In [ ]:
# Process and return original match photo paths
match_photos_paths = process_and_save_match_faces(match_images_folder, finetuned_mtcnn, margin=40)
match_image_shapes = [np.array(image).shape for image in match_photos_all]
print("Shapes of images in the list match_photos_all:", match_image_shapes)
print(f"Number of match photos: {len(match_photos_all)}")

Shapes of images in the list match_photos_all: [(314, 254, 3)]
Number of match photos: 1


In [13]:
# Generate embeddings for match (game) photos
game_embeddings = []
for i, match_photo in enumerate(match_photos_all):
    match_image = np.array(match_photo)  # Convert the PIL image to a NumPy array
    embedding = get_face_embedding(match_image)  # Get embedding
    if embedding is not None:
        game_embeddings.append((i, embedding))  # Store the index and embedding


NameError: name 'get_face_embedding' is not defined

In [14]:
# Compare studio embeddings with match embeddings
results = {}
threshold = 0.4  # Define a similarity threshold for matches

In [18]:
for studio_idx, studio_embedding in studio_embeddings:
    matched_photos = []
    for game_idx, game_embedding in game_embeddings:
        similarity = cosine_similarity_metric(studio_embedding, game_embedding)
        if similarity > threshold:  # If similarity is above the threshold, consider it a match
            matched_photos.append((game_idx, similarity))
    results[studio_idx] = matched_photos 

NameError: name 'game_embeddings' is not defined

In [58]:
# Display results, but load and display the original images (using the file paths)
for studio_idx, matches in results.items():
    print(f'Matching game photos for studio photo index {studio_idx}:')
    print(matches)  # List of matched game photos (indices and distances)
    if matches:
        # Load and display the studio image using the file path
        studio_image = Image.open(studio_photos_paths[studio_idx])
        # Set up the plot for studio and matched game images
        plt.figure(figsize=(20, 6))
        plt.subplot(1, len(matches) + 1, 1)
        plt.imshow(studio_image)
        plt.title(f'Studio Photo Index: {studio_idx}')
        plt.axis('off')
        # Display each matched game image using the original file path
        for i, (game_idx, distance) in enumerate(matches):
            game_image = Image.open(match_photos_paths[game_idx])  # Load the match image by path
            plt.subplot(1, len(matches) + 1, i + 2)
            plt.imshow(game_image)
            plt.title(f'Game Photo Index: {game_idx}\nDistance: {distance:.4f}')
            plt.axis('off')
        # Show the plot with studio and match images
        plt.show()
    else:
        print("No matches found.")